# VGGNet 2014

## 技巧+实现
- 更深
- 多实用3\*3 两个->5 5 三个->7 7
    - 视野域不变
    - 参数减少
    - 非线性变换因为层数增大而增多
- 1\*1 非线性变换
    - 实现通道变换 降维/升维
- 每次经过pooling 通道数目翻倍
- 11->19 参数基本不变
- 递进式训练：从浅层网络-> 深层网络
- 数据增强进一步：多尺度(随机)
    - 256 256 -> 放大 之后对于不同尺度进行抽样224\*224 不同尺度对应不同分类器 最后ensemble
- 与AlexNet 很相似 全连接层仍然在
- A-LRN版本沿用Alex中的局部归一化 通道与相邻通道进行归一化

### data prepocessing

In [1]:
import tensorflow as tf
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

CIFAR_DIR = './cifar-10-batches-py'
print(os.listdir(CIFAR_DIR))

['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


In [2]:
def load_data(filename):
    """read data from data file."""
    with open(filename, 'rb') as f:
        data = pickle.load(f, encoding='bytes')
        return data[b'data'], data[b'labels']

# tensorflow.Dataset.
class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        # hstack vstack 用于合并二维数组变成矩阵形式
        self._data = np.vstack(all_data)
        self._data = self._data / 127.5 - 1
        # 小心
        self._labels = np.hstack(all_labels)
        print(self._data.shape)
        print(self._labels.shape)
        
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
            
    def _shuffle_data(self):
        # [0,1,2,3,4,5] -> [5,3,2,4,0,1]
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):
        """return batch_size examples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels

train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

(50000, 3072)
(50000,)
(10000, 3072)
(10000,)


### model
- 可以达到70%

In [3]:
tf.reset_default_graph() # 利用这个可清空defualt graph以及nodes

x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int64, [None])
# [None], eg: [0,5,6,3]
x_image = tf.reshape(x, [-1, 3, 32, 32])
# 32*32
x_image = tf.transpose(x_image, perm=[0, 2, 3, 1])

# conv1: 神经元图， feature_map, 输出图像
conv1_1 = tf.layers.conv2d(x_image,
                         32, # output channel number
                         (3,3), # kernel size
                         padding = 'same',
                         activation = tf.nn.relu,
                         name = 'conv1_1')

conv1_2 = tf.layers.conv2d(conv1_1,
                         32, # output channel number
                         (3,3), # kernel size
                         padding = 'same',
                         activation = tf.nn.relu,
                         name = 'conv1_2')
# 16 * 16
pooling1 = tf.layers.max_pooling2d(conv1_2,
                                   (2, 2), # kernel size
                                   (2, 2), # stride
                                   name = 'pool1')


conv2_1 = tf.layers.conv2d(pooling1,
                         64, # output channel number
                         (3,3), # kernel size
                         padding = 'same',
                         activation = tf.nn.relu,
                         name = 'conv2_1')
conv2_2 = tf.layers.conv2d(conv2_1,
                         64, # output channel number
                         (3,3), # kernel size
                         padding = 'same',
                         activation = tf.nn.relu,
                         name = 'conv2_2')
# 8 * 8
pooling2 = tf.layers.max_pooling2d(conv2_2,
                                   (2, 2), # kernel size
                                   (2, 2), # stride
                                   name = 'pool2')

conv3_1 = tf.layers.conv2d(pooling2,
                         128, # output channel number
                         (3,3), # kernel size
                         padding = 'same',
                         activation = tf.nn.relu,
                         name = 'conv3_1')
conv3_2 = tf.layers.conv2d(conv3_1,
                         128, # output channel number
                         (3,3), # kernel size
                         padding = 'same',
                         activation = tf.nn.relu,
                         name = 'conv3_2')

# 4 * 4 * 32
pooling3 = tf.layers.max_pooling2d(conv3_2,
                                   (2, 2), # kernel size
                                   (2, 2), # stride
                                   name = 'pool3')
# [None, 4 * 4 * 32]
flatten = tf.layers.flatten(pooling3)
y_ = tf.layers.dense(flatten, 10)

loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)
# y_ -> sofmax
# y -> one_hot
# loss = ylogy_

# [1,0,1,1,1,0,0,0]
correct_prediction = tf.equal(tf.argmax(y_, 1), y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

### train 

In [4]:
# cfg = tf.ConfigProto(allow_soft_placement=True )
# cfg.gpu_options.allow_growth = True
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 1000
test_steps = 100

# train 10k: 71.35%
with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run(
            [loss, accuracy, train_op],
            feed_dict={
                x: batch_data,
                y: batch_labels})
        if (i+1) % 50 == 0:
            print('[Train] Step: %d, loss: %4.5f, acc: %4.5f' 
                  % (i+1, loss_val, acc_val))
        if (i+1) % 200 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels \
                    = test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict = {
                        x: test_batch_data, 
                        y: test_batch_labels
                    })
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test ] Step: %d, acc: %4.5f'
                  % (i+1, test_acc))

[Train] Step: 50, loss: 1.96816, acc: 0.40000
[Train] Step: 100, loss: 1.90950, acc: 0.15000
[Train] Step: 150, loss: 2.09852, acc: 0.20000
[Train] Step: 200, loss: 1.92543, acc: 0.30000
(10000, 3072)
(10000,)
[Test ] Step: 200, acc: 0.32950
[Train] Step: 250, loss: 1.61349, acc: 0.40000
[Train] Step: 300, loss: 1.99049, acc: 0.40000
[Train] Step: 350, loss: 1.56420, acc: 0.45000
[Train] Step: 400, loss: 1.94742, acc: 0.35000
(10000, 3072)
(10000,)
[Test ] Step: 400, acc: 0.39350
[Train] Step: 450, loss: 1.10787, acc: 0.60000
[Train] Step: 500, loss: 1.64498, acc: 0.50000
[Train] Step: 550, loss: 1.61738, acc: 0.20000
[Train] Step: 600, loss: 1.49076, acc: 0.40000
(10000, 3072)
(10000,)
[Test ] Step: 600, acc: 0.43450
[Train] Step: 650, loss: 1.54558, acc: 0.40000
[Train] Step: 700, loss: 1.26109, acc: 0.50000
[Train] Step: 750, loss: 1.16634, acc: 0.55000
[Train] Step: 800, loss: 1.27678, acc: 0.50000
(10000, 3072)
(10000,)
[Test ] Step: 800, acc: 0.47900
[Train] Step: 850, loss: 1.37